# Germanic (Just English)

In [ ]:
# Required imports
import pandas as pd
import openai
import json
import os
import re

# Define the prompting method and language family
language_family = "germanic"
method = "direct_inference"
model = "gpt-3.5-turbo"

# Load data
input_file_path = f'JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

openai.api_key = 'sk-proj-SkTnPbxtFYJhTllYa6ZJT3BlbkFJmWpwsMmUGdkJjAY9EWvc'

def get_gpt_response(claim, model=model):
    system_prompt = ""
    user_prompt = f"'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    # Remove non-alphanumeric characters except spaces
    cleaned_output = re.sub(r'[^a-zA-Z\s]', '', output)
    return cleaned_output

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    output = get_gpt_response(claim)
    
    # Print the model's output
    print(f"Model Output: {output}")
    
    # Clean the output
    cleaned_output = clean_output(output)
    
    # Extract final answer from the cleaned output
    final_answer = None
    if "final answer true" in cleaned_output.lower():
        final_answer = "true"
    elif "final answer false" in cleaned_output.lower():
        final_answer = "false"
    elif "final answer mostly true" in cleaned_output.lower():
        final_answer = "mostly true"
    elif "final answer mostly false" in cleaned_output.lower():
        final_answer = "mostly false"
    elif "final answer half true" in cleaned_output.lower():
        final_answer = "half true"
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


# Romance

In [ ]:
# Required imports
import pandas as pd
import openai
import json
import os
import re

# Define the prompting method and language family
language_family = "romance"
method = "direct_inference"
model = "gpt-3.5-turbo"

# Load data
input_file_path = f'JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

openai.api_key = 'sk-proj-SkTnPbxtFYJhTllYa6ZJT3BlbkFJmWpwsMmUGdkJjAY9EWvc'

# Language-specific user prompts and final answer mappings
prompts = {
    "es": "'{claim}' ¿Es esta afirmación «Verdadera», «Casi verdadera», «Medio verdadera», «Casi falsa» o «Falsa»? Asegúrese siempre de que su respuesta final lleva el prefijo «Respuesta final:» seguido de «Verdadero», «Casi verdadero», «Medio verdadero», «Casi falso» o «Falso».",
    "it": "'{claim}' Questa affermazione è “vera”, “quasi vera”, “mezza vera”, “quasi falsa” o “falsa”? La risposta finale deve essere sempre preceduta da “Risposta finale:” seguita da “Vero”, “Quasi vero”, “Mezzo vero”, “Quasi falso” o “Falso”.",
    "pt": "'{claim}' Esta afirmação é “Verdadeira”, “Quase Verdadeira”, “Meia Verdadeira”, “Quase Falsa” ou “Falsa”? Deve certificar-se sempre de que a sua resposta final tem o prefixo “Resposta Final:” seguido de “Verdadeiro”, “Quase Verdadeiro”, “Meio Verdadeiro”, “Quase Falso” ou “Falso”.",
    "fr": "'{claim}' Cette affirmation est-elle « vraie », « en grande partie vraie », « à moitié vraie », « en grande partie fausse » ou « fausse » ? Vous devez toujours vous assurer que votre réponse finale est précédée de la mention « Réponse finale: » suivie de « Vrai », « Plus ou moins vrai », « Moitié vrai », « Plus ou moins faux » ou « Faux ».",
    "ro": "'{claim}' Această afirmație este „adevărată”, „în mare parte adevărată”, „pe jumătate adevărată”, „în mare parte falsă” sau „falsă”? Trebuie să vă asigurați întotdeauna că răspunsul dumneavoastră final este precedat de „Răspuns final:” urmat de „Adevărat”, „În mare parte adevărat”, „Pe jumătate adevărat”, „În mare parte fals” sau „Fals”."
}

final_answer_mappings = {
    "es": {"respuesta final: verdadero": "true", "respuesta final: casi verdadero": "mostly true", "respuesta final: medio verdadero": "half true", "respuesta final: casi falso": "mostly false", "respuesta final: falso": "false",
           "verdadero": "true", "casi verdadero": "mostly true", "medio verdadero": "half true", "casi falso": "mostly false", "falso": "false"},
    "it": {"risposta finale: vero": "true", "risposta finale: quasi vero": "mostly true", "risposta finale: mezzo vero": "half true", "risposta finale: quasi falso": "mostly false", "risposta finale: falso": "false",
           "vero": "true", "quasi vero": "mostly true", "mezzo vero": "half true", "quasi falso": "mostly false", "falso": "false"},
    "pt": {"resposta final: verdadeiro": "true", "resposta final: quase verdadeiro": "mostly true", "resposta final: meio verdadeiro": "half true", "resposta final: quase falso": "mostly false", "resposta final: falso": "false",
           "verdadeiro": "true", "quase verdadeiro": "mostly true", "meio verdadeiro": "half true", "quase falso": "mostly false", "falso": "false"},
    "fr": {"réponse finale: vrai": "true", "réponse finale: plus ou moins vrai": "mostly true", "réponse finale: moitié vrai": "half true", "réponse finale: plus ou moins faux": "mostly false", "réponse finale: faux": "false",
           "vrai": "true", "plus ou moins vrai": "mostly true", "moitié vrai": "half true", "plus ou moins faux": "mostly false", "faux": "false"},
    "ro": {"răspuns final: adevărat": "true", "răspuns final: în mare parte adevărat": "mostly true", "răspuns final: pe jumătate adevărat": "half true", "răspuns final: în mare parte fals": "mostly false", "răspuns final: fals": "false",
           "adevărat": "true", "în mare parte adevărat": "mostly true", "pe jumătate adevărat": "half true", "în mare parte fals": "mostly false", "fals": "false"}
}

def get_gpt_response(claim, language, model=model):
    system_prompt = ""
    
    if language not in prompts:
        raise ValueError(f"Unsupported language: {language}")

    user_prompt = prompts[language].format(claim=claim)

    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Function to clean the output text
def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    output = get_gpt_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    
    # Extract final answer from the cleaned output using language-specific mappings
    final_answer = None
    for keyword, answer in final_answer_mappings[language].items():
        if keyword in cleaned_output:
            final_answer = answer
            break
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")
